In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('../src/data/fiinx/raw_data.csv')
df

,Date,DXG,TCH,HDG,HDC,KDH,IDC,KBC,DPG,VHM,NLG
0,12/27/2024,"15,700","15,450","29,050","25,150","35,800","55,900","27,650","46,350","40,350","36,400"
1,12/26/2024,"16,000","15,650","29,800","25,350","35,850","56,200","27,550","47,750","40,650","36,200"
2,12/25/2024,"16,300","15,600","29,900","25,350","35,900","56,300","28,150","48,050","40,600","36,300"
3,12/24/2024,"16,450","15,100","30,000","25,300","35,950","56,000","27,600","47,700","40,350","35,700"
4,12/23/2024,"17,650","15,300","30,800","26,100","35,900","56,200","27,800","50,200","40,600","36,250"
...,...,...,...,...,...,...,...,...,...,...,...
1246,01/03/2020,"13,900","33,700","30,650","24,500","26,400","18,300","15,300","39,600","85,200","27,000"
1247,01/02/2020,"14,300","33,300","30,000","24,500","26,500","18,400","15,200","40,700","84,900","27,500"
1248,12/31/2019,"14,500","32,650","30,000","23,800","26,900","18,900","15,450","41,000","84,800","27,500"
1249,12/30/2019,"14,650","32,100","30,150","23,900","27,000","18,900","15,450","41,300","85,500","27,500"


In [ ]:

# config.py
from typing import List, Dict

INDICES: List[str] = [
    'IonQ', 'Rigetti Computing', 'Quantum Computing Inc.',
    'D-Wave Quantum', 'Alphabet', 'IBM',
    'Microsoft', 'Nvidia', 'Defiance Quantum ETF', 'Global X Future Analytics Tech'
]

SYMBOLS: List[str] = [
    'IONQ', 'RGTI', 'QUBT', 'QBTS', 'GOOGL', 
    'IBM', 'MSFT', 'NVDA', 'QTUM', 'AIQ'
]

# Model configuration
SEQUENCE_LENGTH: int = 63
HIDDEN_SIZE: int = 32
NUM_LAYERS: int = 4
BATCH_SIZE: int = 32
LEARNING_RATE: float = 0.001
EPOCHS: int = 100
FORECAST_DAYS: int = 63  # 3 months
"""

"""
# data_loader.py
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
from typing import Dict, Tuple

class StockDataProcessor:
    def __init__(self, csv_path: str):
        self.df = self._load_data(csv_path)
        self.scalers = {}
        
    def _load_data(self, csv_path: str) -> pd.DataFrame:
        df = pd.read_csv(csv_path)
        df = df.dropna(axis=0)
        df.columns = ['Date'] + SYMBOLS
        return df.reset_index(drop=True)
    
    def prepare_data(self, symbol: str) -> Dict:
        data = self.df[symbol].values.reshape(-1, 1)
        self.scalers[symbol] = MinMaxScaler()
        data_normalized = self.scalers[symbol].fit_transform(data)
        
        return self._create_sequences(data_normalized, symbol)
    
    def _create_sequences(self, data: np.ndarray, symbol: str) -> Dict:
        X, y = [], []
        for i in range(len(data) - SEQUENCE_LENGTH):
            X.append(data[i:i+SEQUENCE_LENGTH])
            y.append(data[i+SEQUENCE_LENGTH])
            
        X, y = np.array(X), np.array(y)
        return self._split_and_convert(X, y, symbol)
    
    def _split_and_convert(self, X: np.ndarray, y: np.ndarray, symbol: str) -> Dict:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        X_train = torch.tensor(X_train, dtype=torch.float32)
        y_train = torch.tensor(y_train, dtype=torch.float32)
        X_test = torch.tensor(X_test, dtype=torch.float32)
        y_test = torch.tensor(y_test, dtype=torch.float32)
        
        return {
            'train_loader': DataLoader(
                list(zip(X_train, y_train)), 
                batch_size=BATCH_SIZE, 
                shuffle=True
            ),
            'X_test': X_test,
            'y_test': y_test,
            'scaler': self.scalers[symbol]
        }
"""


"""
# model.py
import torch
import torch.nn as nn

class StockRNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int, output_size: int):
        super(StockRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, (h0, c0))
        return self.fc(out[:, -1, :])
"""

"""
# trainer.py
import torch
import torch.nn as nn
from typing import Dict
import numpy as np
from tqdm import tqdm

class ModelTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, optimizer: torch.optim.Optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
    
    def train_model(self, train_loader: torch.utils.data.DataLoader) -> None:
        for epoch in range(EPOCHS):
            self.model.train()
            epoch_loss = 0
            for batch_X, batch_y in train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            
            print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss/len(train_loader):.4f}')
    
    def predict(self, X_test: torch.Tensor) -> np.ndarray:
        self.model.eval()
        with torch.no_grad():
            return self.model(X_test).numpy()
    
    def predict_future(self, last_sequence: np.ndarray) -> np.ndarray:
        self.model.eval()
        future_predictions = []
        temp_sequence = np.copy(last_sequence)
        
        for _ in range(FORECAST_DAYS):
            with torch.no_grad():
                future_pred = self.model(
                    torch.tensor(temp_sequence, dtype=torch.float32)
                ).numpy()
                future_predictions.append(future_pred[0, 0])
                temp_sequence = np.append(
                    temp_sequence[:, 1:, :],
                    future_pred.reshape(1, 1, 1),
                    axis=1
                )
        
        return np.array(future_predictions).reshape(-1, 1)
"""


"""
# visualization.py
import matplotlib.pyplot as plt
import numpy as np

class StockVisualizer:
    @staticmethod
    def plot_predictions(
        actual: np.ndarray,
        predicted: np.ndarray,
        future_predicted: np.ndarray,
        symbol: str
    ) -> None:
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        
        # Test set predictions
        axes[0].plot(actual, label='Actual Prices', color='blue')
        axes[0].plot(predicted, label='Predicted Prices', marker='o', alpha=0.5, color='red')
        axes[0].legend()
        axes[0].set_title(f'{symbol} Price Prediction (Test Set)', weight='bold')
        axes[0].set_xlabel('Time [days]', fontsize=12, weight='bold')
        axes[0].set_ylabel('Value [USD]', fontsize=12, weight='bold')
        axes[0].grid(True)
        
        # Future predictions
        axes[1].plot(future_predicted, label='Predicted Future Prices', color='red', alpha=0.8)
        axes[1].legend()
        axes[1].set_title(f'{symbol} 3-Month Price Forecast', weight='bold')
        axes[1].set_xlabel('Time [days]', fontsize=12, weight='bold')
        axes[1].set_ylabel('Value [USD]', fontsize=12, weight='bold')
        axes[1].grid(True)
        
        plt.tight_layout()
        plt.savefig(f'{symbol}.jpg', dpi=600)
        plt.close()
"""


"""
# main.py
import torch
import torch.nn as nn
from config import *
from data_loader import StockDataProcessor
from model import StockRNN
from trainer import ModelTrainer
from visualization import StockVisualizer

def process_symbol(symbol: str, data_processor: StockDataProcessor) -> None:
    # Prepare data
    data_dict = data_processor.prepare_data(symbol)
    
    # Initialize model
    model = StockRNN(
        input_size=1,
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
        output_size=1
    )
    
    # Initialize trainer
    trainer = ModelTrainer(
        model=model,
        criterion=nn.MSELoss(),
        optimizer=torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    )
    
    # Train model
    print(f"\nTraining model for {symbol}")
    trainer.train_model(data_dict['train_loader'])
    
    # Make predictions
    predictions = trainer.predict(data_dict['X_test'])
    y_test_np = data_dict['y_test'].numpy()
    
    # Inverse transform predictions
    predictions = data_dict['scaler'].inverse_transform(predictions)
    y_test_np = data_dict['scaler'].inverse_transform(y_test_np)
    
    # Future predictions
    last_sequence = data_dict['X_test'][-2].reshape(1, SEQUENCE_LENGTH, 1)
    future_predictions = trainer.predict_future(last_sequence)
    future_predictions = data_dict['scaler'].inverse_transform(future_predictions)
    
    # Visualize results
    StockVisualizer.plot_predictions(
        actual=y_test_np,
        predicted=predictions,
        future_predicted=future_predictions,
        symbol=symbol
    )

def main():
    data_processor = StockDataProcessor('quantum_technology_indices_prices.csv')
    
    for symbol in SYMBOLS:
        process_symbol(symbol, data_processor)

if __name__ == "__main__":
    main()
"""